In [1]:
from skimage.filters import threshold_local,threshold_otsu,try_all_threshold,threshold_mean,threshold_yen
import numpy as np
import cv2
import imutils
from skimage.util.shape import view_as_windows
from skimage.morphology import skeletonize

In [2]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    n=60
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]+n
    rect[2] = pts[np.argmax(s)]-n

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]-n
    #x,y
    rect[1][0]-=n
    rect[1][1]+=n
    rect[3] = pts[np.argmax(diff)]
    #x,y
    rect[3][0]+=n
    rect[3][1]-=n
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
 
 
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
  [0, 0],
  [maxWidth - 1, 0],
  [maxWidth - 1, maxHeight - 1],
  [0, maxHeight - 1]], dtype = "float32")
 
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
    return warped

In [8]:
img_dir='input/10.jpeg'
MORPH = 9
CANNY = 84
HOUGH = 25
orig = cv2.imread(img_dir)

img = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
img_copy = img.copy()
cv2.imwrite("orig.png",img)

cv2.GaussianBlur(img, (3,3), 0, img)
# Remove small details
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(MORPH,MORPH))
dilated = cv2.dilate(img, kernel)
cv2.imwrite("dilated.png",dilated)

edges = cv2.Canny(dilated, 0, CANNY, apertureSize=3)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
edges = cv2.dilate(edges, kernel)
edges = cv2.Canny(edges, 0, CANNY, apertureSize=3)
cv2.imwrite("edges.png",edges)

True

In [9]:
# finding contours
im,contours,heirarchy = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
maxArea=0
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    if(w*h >= maxArea):
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.05 * peri, True)
        maxArea=w*h
cv2.drawContours(orig,[approx], -1, (0, 255, 0), 1)
cv2.imwrite('outline.png',orig)


True

In [10]:
ratio=1
warped = four_point_transform(img_copy, approx.reshape(len(approx), 2) * ratio)
cv2.imwrite('result.png',warped)


True

In [13]:
img_gray = cv2.imread('input/11.png',0)
dilated_img = cv2.dilate(img_gray, np.ones((7,7), np.uint8)) 
bg_img = cv2.medianBlur(dilated_img, 21)
diff_img = 255 - cv2.absdiff(img_gray, bg_img)
cv2.imwrite("shadowOut.png",diff_img)
norm_img = diff_img.copy() # Needed for 3.x compatibility
cv2.normalize(diff_img, norm_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
_, thr_img = cv2.threshold(norm_img, 230, 0, cv2.THRESH_TRUNC)
cv2.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
cv2.imwrite("thres.png",thr_img)
thres =threshold_yen(thr_img)
final = (thr_img >thres)*255
cv2.imwrite("final.png",final)
#fig, ax = try_all_threshold(thr_img, figsize=(20, 30), verbose=False)


True

In [21]:
#try to remove any opening in shapes
kernels = [cv2.getStructuringElement(cv2.MORPH_RECT,(1,4))]
kernels.append(np.array([
       [1,0, 0, 0],
       [0,1, 0, 0],
       [0,0, 1, 0],
       [0,0, 0, 1]], dtype=np.uint8))
kernels.append(np.array([
       [0, 0,0, 1],
       [0, 0, 1,0],
       [0, 1, 0,0],
       [1, 0, 0,0]], dtype=np.uint8))
kernels.append(cv2.getStructuringElement(cv2.MORPH_RECT,(4,1)))   
sk=((255-final)/255).astype(np.uint8)
sk = skeletonize(sk)
cv2.imwrite("sk.png",sk*255)
for kernel in kernels:
    opening=(255- sk*255).astype(np.uint8)
    opening = cv2.morphologyEx(opening, cv2.MORPH_OPEN, kernel,iterations=3)
    sk=((255-opening)/255).astype(np.uint8)
    sk = skeletonize(sk)
cv2.imwrite("opened.png",opening)
opening=(1 - opening/255).astype(np.uint8)
final_bin=(1 - final/255).astype(np.uint8)
cv2.imwrite("test1.png",opening*255)
cv2.imwrite("test2.png",final_bin*255)

connect = opening | final_bin
cv2.imwrite("connect.png",255 - connect*255)


True